In [5]:
import numpy as np
import os
from random import shuffle
import re
import urllib.request
import zipfile
import lxml.etree
import pandas as pd
#download the data
x = pd.read_csv('/Users/randy/Documents/GitHub/Twitch_Chat_Harassment/toxic_comment_classification/jigsaw-toxic-comment-classification-challenge/cleaned_train.csv')
input_text = x

In [6]:
#download the data


In [7]:
# remove parenthesis 
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)
# store as list of sentences
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)
# store as list of lists of words
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

TypeError: expected string or bytes-like object

Now, sentences_ted has been transformed into a two dimensional array with each element being a word. Let’s print out the first and the second element.

In [ ]:
from gensim.models import Word2Vec
model_ted = Word2Vec(sentences=sentences_ted, size=100, window=5, min_count=5, workers=4, sg=0)


In [ ]:
model_ted.wv.most_similar("man")

Although Word2Vec successfully handles the issue posed by one-hot vector, it has several limitation. The biggest challenge is that it is not able to represent words that do not appear in the training dataset. Even though using a larger training set that contains more vocabulary, some rare words used very seldom can never be mapped to vectors.

# Now for FastText

FastText is an extension to Word2Vec proposed by Facebook in 2016. Instead of feeding individual words into the Neural Network, FastText breaks words into several n-grams (sub-words). For instance, the tri-grams for the word apple is app, ppl, and ple (ignoring the starting and ending of boundaries of words). The word embedding vector for apple will be the sum of all these n-grams. After training the Neural Network, we will have word embeddings for all the n-grams given the training dataset. Rare words can now be properly represented since it is highly likely that some of their n-grams also appears in other words. I will show you how to use FastText with Gensim in the following section.

In [ ]:
from gensim.models import FastText
model_ted = FastText(sentences_ted, size=100, window=5, min_count=5, workers=4,sg=1)

Let’s try it with the word Gastroenteritis, which is rarely used and does not appear in the training dataset.

In [ ]:
model_ted.wv.most_similar("Gastroenteritis")

Even though the word Gastroenteritis does not exist in the training dataset, it is still capable of figuring out this word is closely related to some medical terms. If we try this in the Word2Vec defined previously, it would pop out error because such word does not exist in the training dataset. Although it takes longer time to train a FastText model (number of n-grams > number of words), it performs better than Word2Vec and allows rare words to be represented appropriately.

# Second attempt at useful fastText

In [27]:
import fastText
 
# Skipgram model
model = fastText.skipgram('data.txt', 'model')

AttributeError: module 'fastText' has no attribute 'skipgram'

In [25]:
import fasttext
 
# CBOW model
model = fasttext.cbow('TrainingData.txt', 'model')
print(model.words) # list of words in dictionary
 
print(model['machine']) # get the vector of the word 'machine'

ModuleNotFoundError: No module named 'fasttext'

In [12]:
consumercompliants.head()

,Product,Consumer_complaint_narrative
33524,__label__Credit_reporting __label__credit_repa...,I am a victim of identity theft. My personal i...
33700,__label__Credit_reporting __label__credit_repa...,I have reached out to All 3 credit bureaus and...
33728,__label__Credit_reporting __label__credit_repa...,I am a victim of identity theft. My personal i...
33928,__label__Credit_reporting __label__credit_repa...,On all credit agencies the same accounts are n...
35277,__label__Credit_reporting __label__credit_repa...,I have a student loan thats discharge and its ...


In [13]:
wc consumer.complaints.txt

SyntaxError: invalid syntax (<ipython-input-13-60c8e79d3b55>, line 1)

In [14]:
pwd

'/Users/randy/Documents/GitHub/Twitch_Chat_Harassment/notebooks'